In [1]:
import time
import keras

from keras.models import Model
from keras.layers import Dropout, Flatten, BatchNormalization, TimeDistributed, Input, Add, Concatenate
from keras.layers import Dense, Conv2D, MaxPooling2D, LSTM, TimeDistributed, Reshape
import keras.backend as K
import keras.callbacks as callbacks

import pandas as pd
import numpy as np
from numpy import array
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

In [2]:
save_path = "/Users/Brody1/Dropbox/Northwestern/DNA_Cyclizability/benchmarks/"
model_name = "ir_lstm_post_smoothed_5bp_DNAcycP_chrv"
kf = KFold(n_splits = 10, shuffle =True)
num_epochs = 60

In [3]:
def model_cycle():
    inputs = Input(shape=(50, 4, 1))
        
    x = Conv2D(48, kernel_size=(3,4),
                   activation='relu',
                   padding='valid')(inputs)
    x = MaxPooling2D((2,1),padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    # parallel line 1
    fx1 = Conv2D(48, kernel_size=(3,1),
                   activation='relu',
                   padding='same')(x)
    fx1 = BatchNormalization()(fx1)
    fx1 = Dropout(0.2)(fx1)
    fx1 = Conv2D(48, kernel_size=(3,1),
                   activation='relu',
                   padding='same')(fx1)
    fx1 = MaxPooling2D((2,1),padding='same')(fx1)
    fx1 = BatchNormalization()(fx1)
    fx1 = Dropout(0.2)(fx1)
    
    # parallel line 2
    fx2 = Conv2D(48, kernel_size=(11,1),
                   activation='relu',
                   padding='same')(x)
    fx2 = BatchNormalization()(fx2)
    fx2 = Dropout(0.2)(fx2)
    fx2 = Conv2D(48, kernel_size=(21,1),
                   activation='relu',
                   padding='same')(fx2)
    fx2 = MaxPooling2D((2,1),padding='same')(fx2)
    fx2 = BatchNormalization()(fx2)
    fx2 = Dropout(0.2)(fx2)
    
    # Add
    x1 = Concatenate(axis=-3)([fx1, fx2])
    x = Add()([x, x1])
    x = MaxPooling2D((2,1),padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    x = Reshape((K.int_shape(x)[1], K.int_shape(x)[3]))(x)
    x = LSTM(20, return_sequences=False)(x)
    x = Dropout(0.2)(x)

    outputs = Dense(1, activation='linear')(x)
    network = Model(inputs, outputs)
    network.compile(optimizer='rmsprop',
                    loss='mean_squared_error')
    return network

In [4]:
def dnaOneHot(sequence):
    seq_array = array(list(sequence))
    code = {"A": [0], "C": [1], "G": [2], "T": [3], "N": [4],
            "a": [0], "c": [1], "g": [2], "t": [3], "n": [4]}
    onehot_encoded_seq = []
    for char in seq_array:
        onehot_encoded = np.zeros(5)
        onehot_encoded[code[char]] = 1
        onehot_encoded_seq.append(onehot_encoded[0:4])
    return onehot_encoded_seq

class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.process_time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.process_time() - self.epoch_time_start)

In [15]:
data_cerevisiae_nucle = pd.read_csv("/Users/Brody1/Dropbox/Northwestern/DNA_Cyclizability/cycle1.txt",delimiter = ",")
X1 = []
for sequence_nt in data_cerevisiae_nucle["Sequence"]:
    X1.append(dnaOneHot(sequence_nt))
X1 = array(X1)
X1 = X1.reshape((X1.shape[0],50,4,1))
X1_reverse = np.flip(X1,[1,2])
Y1_C0 = data_cerevisiae_nucle["C0"].values.astype(float)
m1 = np.mean(Y1_C0)
std1 = np.std(Y1_C0)
Z1_C0 = (Y1_C0-m1)/std1

In [16]:
data_random_library = pd.read_csv("/Users/Brody1/Dropbox/Northwestern/DNA_Cyclizability/cycle3.txt",delimiter = ",")
X3 = []
for sequence_nt in data_random_library["Sequence"]:
    X3.append(dnaOneHot(sequence_nt))
X3 = array(X3)
X3 = X3.reshape((X3.shape[0],50,4,1))
X3_reverse = np.flip(X3,[1,2])
Y3_C0 = data_random_library["C0"].values.astype(float)
m3 = np.mean(Y3_C0)
std3 = np.std(Y3_C0)
Z3_C0 = (Y3_C0-m3)/std3

In [17]:
data_tiling = pd.read_csv("/Users/Brody1/Dropbox/Northwestern/DNA_Cyclizability/cycle5.txt",delimiter = ",")
X5 = []
for sequence_nt in data_tiling["Sequence"]:
    X5.append(dnaOneHot(sequence_nt))
X5 = array(X5)
X5 = X5.reshape((X5.shape[0],50,4,1))
X5_reverse = np.flip(X5,[1,2])
Y5_C0 = data_tiling["C0"].values.astype(float)
m5 = np.mean(Y5_C0)
std5 = np.std(Y5_C0)
Z5_C0 = (Y5_C0-m5)/std5

In [37]:
data_chr5 = pd.read_csv("/Users/Brody1/Dropbox/Northwestern/DNA_Cyclizability/data/Created/yeast_chrV_ir_lstm_tiling_post_smoothed_matched.csv")
# Remove first row:
data_chr5 = data_chr5.iloc[1:(data_chr5.shape[0]-1),]
print(data_chr5.shape)
X6 = []
for sequence_nt in data_chr5["Sequence"]:
    X6.append(dnaOneHot(sequence_nt))
X6 = array(X6)
X6 = X6.reshape((X6.shape[0],50,4,1))
X6_reverse = np.flip(X6,[1,2])
Y6 = data_chr5["DNAcycP_pred_chrV_post_smooth_5bp"].values.astype(float)
Y6_C0 = data_chr5["C0"].values.astype(float)
m6 = np.mean(Y6)
std6 = np.std(Y6)
Z6 = (Y6-m6)/std6

(82277, 32)


In [7]:
# #### chrv

# VALIDATION_LOSS = []
# fold_var = 1
# n = Z6.shape[0]

# fits = []
# detrend = []
# times = []
# times2 = []

# for train_index, val_index in kf.split(Z6):
#     training_X = X6[train_index]
#     training_X_reverse = X6_reverse[train_index]
#     validation_X = X6[val_index]
#     validation_X_reverse = X6_reverse[val_index]
#     training_Y = Z6[train_index]
#     validation_Y = Z6[val_index]
#     # CREATE NEW MODEL
#     model = model_cycle()
#     # CREATE CALLBACKS
#     checkpoint = callbacks.ModelCheckpoint(save_path + model_name+"_chrv_"+str(fold_var)+".h5",
#                                                     monitor='val_loss', verbose=1,
#                                                     save_best_only=True, mode='min')
#     time_callback = TimeHistory()

#     history = model.fit(training_X, training_Y,
#                         epochs=num_epochs,
#                         callbacks= [checkpoint, time_callback],
#                         validation_data=(validation_X, validation_Y))
#     model.load_weights(save_path + model_name+"_chrv_"+str(fold_var)+".h5")
#     model.save(save_path+ model_name+"_chrv_"+str(fold_var),save_traces=False)
#     times.append(time_callback.times)

#     pred_Y = model.predict(training_X)
#     pred_Y = pred_Y.reshape(pred_Y.shape[0])
#     pred_Y_reverse = model.predict(training_X_reverse)
#     pred_Y_reverse = pred_Y_reverse.reshape(pred_Y_reverse.shape[0])
#     pred_Y = (pred_Y+pred_Y_reverse)/2
#     reg =  LinearRegression().fit(array(pred_Y).reshape(-1, 1), array(training_Y).reshape(-1, 1))
    
#     detrend_int = reg.intercept_
#     detrend_slope = reg.coef_
#     detrend.append([float(detrend_int), float(detrend_slope)])
    
#     start_time = time.process_time()
#     fit = model.predict(validation_X)
#     fit = fit.reshape(fit.shape[0])
#     fit_reverse = model.predict(validation_X_reverse)
#     fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
#     reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
#     fit = (fit + fit_reverse)/2
#     fit = fit.flatten()
#     fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit),reverse_corr]
#     fits.append(fit_tmp)
#     fit = detrend_int + fit * detrend_slope
#     fit = fit.flatten()
#     fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit),reverse_corr]
#     time0 = time.process_time() - start_time
#     times2.append([time0])
#     fits.append(fit_tmp)
    
#     K.clear_session()
#     fold_var += 1

# detrend = array(detrend)
# detrend = pd.DataFrame(detrend)
# detrend.to_csv(save_path +model_name+"_detrend_chrv.txt", index = False)

# fits = array(fits)
# fits = pd.DataFrame((fits))
# fits.to_csv(save_path +model_name+"_fits_chrv.txt", index = False)

# with open(save_path +model_name+"_time_chrv.txt", "w") as file:
#     for row in times:
#         s = " ".join(map(str, row))
#         file.write(s+'\n')

# with open(save_path +model_name+"_pred_time_chrv.txt", "w") as file:
#     for row in times2:
#         s = " ".join(map(str, row))
#         file.write(s+'\n')

In [19]:
model_chrv = keras.models.load_model(save_path + model_name + "_chrv_10.h5")

In [26]:
chrv_pred_nuc = model_chrv.predict(X1)*std1 + m1

623/623 [==============================] - 2s 2ms/step


In [27]:
chrv_pred_random = model_chrv.predict(X3)*std3 + m3

390/390 [==============================] - 1s 2ms/step


In [21]:
chrv_pred_tiling = model_chrv.predict(X5)*std5 + m5

2574/2574 [==============================] - 6s 2ms/step


In [39]:
chrv_pred_chrv = model_chrv.predict(X6)*std6 + m6

2572/2572 [==============================] - 5s 2ms/step


In [48]:
print(f"Correlation between predicted smooth C0 and original C0, nucleosome library: {round(np.corrcoef(array(chrv_pred_nuc).reshape(-1), Y1_C0)[0,1], 3)}")
print(f"\t (compared to 0.893)")

Correlation between predicted smooth C0 and original C0, nucleosome library: 0.868
	 (compared to 0.893)


In [49]:
print(f"Correlation between predicted smooth C0 and original C0, random library: {round(np.corrcoef(array(chrv_pred_random).reshape(-1), Y3_C0)[0,1], 3)}")
print(f"\t (compared to 0.930)")

Correlation between predicted smooth C0 and original C0, random library: 0.896
	 (compared to 0.930)


In [50]:
print(f"Correlation between predicted smooth C0 and original C0, tiling library: {round(np.corrcoef(array(chrv_pred_tiling).reshape(-1), Y5_C0)[0,1], 3)}")
print(f"\t (compared to 0.916)")

Correlation between predicted smooth C0 and original C0, tiling library: 0.886
	 (compared to 0.916)


In [51]:
print(f"Correlation between predicted smooth C0 and original C0, chrV library: {round(np.corrcoef(array(chrv_pred_chrv).reshape(-1), Y6_C0)[0,1], 3)}")
print(f"\t (compared to 0.773)")
print(f"Correlation between smooth C0 and original C0, chrV library: {round(np.corrcoef(Y6, Y6_C0)[0,1], 4)}")

Correlation between predicted smooth C0 and original C0, chrV library: 0.747
	 (compared to 0.773)
Correlation between smooth C0 and original C0, chrV library: 0.751
